### Load Files

In [1]:
from pysph.tools.sph_evaluator import SPHEvaluator
from pysph.base.kernels import WendlandQuinticC4
from pysph.solver.utils import load

In [18]:
data1 = load("outputs\\tgv_re_100_nx_50\\taylor_green_00000.npz")
data2 = load("outputs\\tgv_re_100_nx_50\\taylor_green_00100.npz")

In [21]:
data1['arrays']['fluid'].get('x')

array([0.01166809, 0.0328813 , 0.05000046, ..., 0.95227256, 0.97137368,
       0.99140247])

In [22]:
data1['arrays']['fluid'].get('x')

array([0.01166809, 0.0328813 , 0.05000046, ..., 0.95227256, 0.97137368,
       0.99140247])